In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

In [19]:
# Load YOLOv8 models
cleaf_model = YOLO('cleaf.pt')
cdisease_model = YOLO('cdisease.pt')

In [22]:
# Verify and print the class names
print("Cleaf model classes:", cleaf_model.names)
print("Cdisease model classes:", cdisease_model.names)

Cleaf model classes: {0: 'arabica', 1: 'liberica', 2: 'robusta'}
Cdisease model classes: {0: 'brown_eye_spot', 1: 'leaf_miner', 2: 'leaf_rust', 3: 'red_spider_mite'}


In [61]:
def detect_and_classify_coffee_disease(image_path, output_path=None, conf_threshold=0.25, iou_threshold=0.45, screen_width=1920, screen_height=1080):
    """
    Detect and classify coffee diseases using both cleaf and cdisease models.

    Parameters:
    - image_path (str): Path to the input image file.
    - output_path (str, optional): Path to save the output image with bounding boxes.
    - conf_threshold (float): Confidence threshold for predictions.
    - iou_threshold (float): IOU threshold for non-max suppression.
    - screen_width (int): Width of the screen for display purposes.
    - screen_height (int): Height of the screen for display purposes.

    Returns:
    - img (numpy.ndarray): The image with bounding boxes drawn.
    """
    # Define colors for different classes
    cleaf_colors = {
        0: (0, 255, 0),    # Green for 'arabica'
        1: (0, 255, 255),  # Yellow for 'liberica'
        2: (255, 0, 0)     # Blue for 'robusta'
    }
    cdisease_colors = {
        0: (255, 165, 0),  # Orange for 'brown_eye_spot'
        1: (255, 0, 255),  # Magenta for 'leaf_miner'
        2: (0, 0, 255),    # Red for 'leaf_rust'
        3: (128, 0, 128)   # Purple for 'red_spider_mite'
    }

    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")

    # Stage 1: Detect the coffee leaf
    leaf_results = cleaf_model.predict(source=img, conf=conf_threshold, iou=iou_threshold, verbose=False)
    if len(leaf_results) > 0:
        leaf_boxes = leaf_results[0].boxes
        if len(leaf_boxes) > 0:
            for box in leaf_boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                confidence = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                class_name = cleaf_model.names[class_id]
                color = cleaf_colors.get(class_id, (255, 255, 255))  # Default to white if class not found

                # Draw the bounding box for the leaf
                cv2.rectangle(img, (x1, y1), (x2, y2), color=color, thickness=2)
                label = f"{class_name} {confidence:.2f}"
                put_label_on_image(img, label, x1, y1, color, screen_width)

                # Stage 2: Detect the disease within the leaf region
                leaf_roi = img[y1:y2, x1:x2]
                disease_results = cdisease_model.predict(source=leaf_roi, conf=conf_threshold, iou=iou_threshold, verbose=False)
                if len(disease_results) > 0:
                    disease_boxes = disease_results[0].boxes
                    if len(disease_boxes) > 0:
                        for dbox in disease_boxes:
                            dx1, dy1, dx2, dy2 = dbox.xyxy[0].cpu().numpy().astype(int)
                            dconfidence = dbox.conf[0].cpu().numpy()
                            dclass_id = int(dbox.cls[0].cpu().numpy())
                            dclass_name = cdisease_model.names[dclass_id]
                            dcolor = cdisease_colors.get(dclass_id, (255, 255, 255))  # Default to white if class not found

                            # Adjust disease box coordinates to match the original image
                            dx1 += x1
                            dy1 += y1
                            dx2 += x1
                            dy2 += y1

                            # Draw the bounding box for the disease
                            cv2.rectangle(img, (dx1, dy1), (dx2, dy2), color=dcolor, thickness=2)
                            dlabel = f"{dclass_name} {dconfidence:.2f}"
                            put_label_on_image(img, dlabel, dx1, dy1, dcolor, screen_width)

    # Resize the image to fit the screen if needed
    img_height, img_width = img.shape[:2]
    scaling_factor = min(screen_width / img_width, screen_height / img_height)
    if scaling_factor < 1.0:
        img = cv2.resize(img, (int(img_width * scaling_factor), int(img_height * scaling_factor)))

    # Optionally save the output image
    if output_path:
        cv2.imwrite(output_path, img)

    return img

In [62]:
def put_label_on_image(img, label, x, y, color, screen_width):
    """
    Put a label on the image, adjusting its position if it goes off-screen.

    Parameters:
    - img (numpy.ndarray): Image on which to put the label.
    - label (str): The text label to put on the image.
    - x (int): X coordinate for label placement.
    - y (int): Y coordinate for label placement.
    - color (tuple): Color of the label background.
    - screen_width (int): Width of the screen to prevent overflow.
    """
    font_scale = 1.0
    font_thickness = 2
    (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

    # Adjust label position if it goes off-screen
    if x + label_width > screen_width:
        x = screen_width - label_width - 10  # 10 pixels padding from the edge

    cv2.rectangle(
        img,
        (x, y - label_height - baseline),
        (x + label_width, y),
        color=color,
        thickness=cv2.FILLED
    )
    cv2.putText(
        img,
        label,
        (x, y - baseline),
        cv2.FONT_HERSHEY_SIMPLEX,
        font_scale,
        (0, 0, 0),
        thickness=font_thickness,
        lineType=cv2.LINE_AA
    )

In [ ]:
if __name__ == "__main__":
    input_image_path = 'test_images/lf2.png'
    output_image_path = 'test_output.jpg'

    result_img = detect_and_classify_coffee_disease(
        input_image_path,
        output_path=output_image_path,
        conf_threshold=0.2,
        iou_threshold=0.4,
        screen_width=1920,  # Your screen width
        screen_height=1080  # Your screen height
    )

    # Display the results using OpenCV
    cv2.imshow('Coffee Disease Detection', result_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
